# Random Forest

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn import datasets
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

In [39]:
# WARNINGS 02/06/2018:
# I updated the preprocessing notebook to keep the date as a feature. Run preprocessing again. 
# (Non provate a fare join qui col train.csv perché incasina tutti gli indici)
data = pd.read_csv('preprocessed_train.csv')
data.head()

# random row
data.loc[4342]

AssortmentType                With Non-Food Department
CloudCover                                           5
Date                                        2017-06-17
Events                                            Rain
HasPromotions                                        0
IsHoliday                                            0
Max_Dew_PointC                                      13
Max_Gust_SpeedKm_h                                  26
Max_Humidity                                        94
Max_Sea_Level_PressurehPa                         1021
Max_TemperatureC                                    20
Max_VisibilityKm                                    31
Max_Wind_SpeedKm_h                                  26
Mean_Dew_PointC                                      9
Mean_Humidity                                       64
Mean_Sea_Level_PressurehPa                        1018
Mean_TemperatureC                                   16
Mean_VisibilityKm                                   13
Mean_Wind_

### Some preprocessing again...

In [40]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

In [41]:
# Get the index for which the rolling fails (region 2, from 03/01/2018 to 04/02/2018)
start = pd.to_datetime('3/01/2018', format='%d/%m/%Y')
stop = pd.to_datetime('4/02/2018', format='%d/%m/%Y')

to_eliminate = data.loc[data.Region == 2]
to_eliminate = to_eliminate[to_eliminate.Date > start]
to_eliminate = to_eliminate[to_eliminate.Date < stop]

# I will remove these rows later...

In [42]:
#to_eliminate

In [43]:
# Keep most important features for our target (see FeatureSelection notebook for details)
# I kept also 'Date' because I need it, I will remove it then

selected_features=[
    'Month', #drop then
    'Date', # drop then
    'Region', # drop then
    'NumberOfSales',
    'NumberOfSales_lastmonth', 
    'HasPromotions', 
    'NumberOfSales_yesterday', 
    'NumberOfSales_lastweek',
    'IsOpen_yesterday',
    'DayOfWeek',
    'NearestCompetitor',
    'Week',
    'StoreID',
    'IsHoliday_tomorrow']
df = data[selected_features]
df.head()

,Month,Date,Region,NumberOfSales,NumberOfSales_lastmonth,HasPromotions,NumberOfSales_yesterday,NumberOfSales_lastweek,IsOpen_yesterday,DayOfWeek,NearestCompetitor,Week,StoreID,IsHoliday_tomorrow
0,4,2016-04-02,7,11129.0,186345.0,0,0.0,44106.0,0.0,5,326,13,1000,0.0
1,4,2016-04-05,7,6981.0,182020.0,0,0.0,40680.0,0.0,1,326,14,1000,0.0
2,4,2016-04-06,7,5222.0,189001.0,0,6981.0,39072.0,1.0,2,326,14,1000,0.0
3,4,2016-04-07,7,6509.0,184113.0,0,5222.0,35239.0,1.0,3,326,14,1000,0.0
4,4,2016-04-08,7,6270.0,181603.0,0,6509.0,29841.0,1.0,4,326,14,1000,0.0


### Recover missing values

Maybe we could improve the performance handling these missing values in better ways

In [44]:
df.NumberOfSales_yesterday.fillna(0, inplace=True)

C:\Users\feder\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### Split dataset in train and validation (che io chiamo test)

In [45]:
def split(df, train_fraction):
    start = pd.to_datetime('1/03/2017', format='%d/%m/%Y')
    stop = pd.to_datetime('1/05/2017', format='%d/%m/%Y')
    train = pd.concat([df[df.Date < start],df[df.Date >= stop]])
    test = df.drop(train.index)
    return train, test


In [46]:
# Save dataset with 'Date' feature and all the rows for the rolling 
df_dates = df

# Remove from the dataset the rows from which starting a rolling generates NaN values
df = df.drop(to_eliminate.index)

train_fraction = 21/24

df_train, df_test = split(df, train_fraction)

In [47]:
null_cols = []
print('Features with NaN:')
for col in df_train.columns:
    if df_train[col].isnull().values.any():
        print(col)
        null_cols.append(col)
 

Features with NaN:
NumberOfSales_lastmonth
NumberOfSales_lastweek
IsOpen_yesterday
IsHoliday_tomorrow


In [48]:
       
# getting indices (rows) of all NaN values
inds = pd.isnull(df_train).any(1).nonzero()[0]

# drop all the rows with NaN values
df_train = df_train.drop(df_train.index[inds])

# checking shapes
print('df: ' + str(df_train.shape))

df: (383590, 14)


In [49]:
# Put rows back to train set 
df_train = pd.concat([df_dates.loc[to_eliminate.index], df_train])

In [50]:
# Save dataset with 'Date' feature and all the rows for the rolling 
df_dates = pd.concat([df_train, df_test])

In [51]:
# create dataset and target
ytrain = df_train['NumberOfSales']
Xtrain = df_train.drop(['Date','Region','NumberOfSales'],axis=1)

ytest = df_test['NumberOfSales']
Xtest = df_test.drop(['Date','Region','NumberOfSales'],axis=1)


## Compute the Random Forest

In [15]:
forest = RandomForestRegressor(n_estimators=250, random_state=0, n_jobs=-1)
forest.fit(Xtrain, ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [16]:
# How much good is the model?
forest.score(Xtest,ytest)

0.8958088331519647

## Run the prediction without knowing NumberOfSales_[lastWeek, lastmonth, yesterday]

In [17]:
# FOR DEBUGGING PORPOSE keep only one region in the test set
Xtest = Xtest[df_dates.Region == 4]

C:\Users\feder\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [18]:
Xtest

,Month,NumberOfSales_lastmonth,HasPromotions,NumberOfSales_yesterday,NumberOfSales_lastweek,IsOpen_yesterday,DayOfWeek,NearestCompetitor,Week,StoreID,IsHoliday_tomorrow
7273,3,169951.0,0,5672.0,37610.0,1.0,2,5439,9,1012,0.0
7274,3,169888.0,0,5451.0,36494.0,1.0,3,5439,9,1012,0.0
7275,3,170376.0,0,5957.0,35328.0,1.0,4,5439,9,1012,0.0
7276,3,171593.0,0,6091.0,34471.0,1.0,5,5439,9,1012,0.0
7277,3,166032.0,1,0.0,35992.0,0.0,0,5439,10,1012,0.0
7278,3,169364.0,1,9362.0,39534.0,1.0,1,5439,10,1012,0.0
7279,3,177463.0,1,8099.0,41961.0,1.0,2,5439,10,1012,0.0
7280,3,175753.0,1,7562.0,44072.0,1.0,3,5439,10,1012,0.0
7281,3,174735.0,1,7872.0,45987.0,1.0,4,5439,10,1012,0.0
7282,3,174742.0,0,7153.0,47049.0,1.0,5,5439,10,1012,0.0


In [52]:
# Forget these NumberOfSales values :(

Xtest['NumberOfSales_yesterday'] = np.NaN
Xtest['NumberOfSales_lastweek'] = np.NaN
Xtest['NumberOfSales_lastmonth'] = np.NaN

In [53]:
# remove all the values about NumberOfSales of the test set
df_dates.loc[Xtest.index,'NumberOfSales_yesterday'] = np.NaN
df_dates.loc[Xtest.index,'NumberOfSales_lastweek'] = np.NaN
df_dates.loc[Xtest.index,'NumberOfSales_lastmonth'] = np.NaN
df_dates.loc[Xtest.index,'NumberOfSales'] = np.NaN

In [54]:
df_dates.loc[Xtest.index]

,Month,Date,Region,NumberOfSales,NumberOfSales_lastmonth,HasPromotions,NumberOfSales_yesterday,NumberOfSales_lastweek,IsOpen_yesterday,DayOfWeek,NearestCompetitor,Week,StoreID,IsHoliday_tomorrow
276,3,2017-03-01,7,NaN,NaN,0,NaN,NaN,1.0,2,326,9,1000,0.0
277,3,2017-03-02,7,NaN,NaN,0,NaN,NaN,1.0,3,326,9,1000,0.0
278,3,2017-03-03,7,NaN,NaN,0,NaN,NaN,1.0,4,326,9,1000,0.0
279,3,2017-03-04,7,NaN,NaN,0,NaN,NaN,1.0,5,326,9,1000,0.0
280,3,2017-03-06,7,NaN,NaN,1,NaN,NaN,0.0,0,326,10,1000,0.0
281,3,2017-03-07,7,NaN,NaN,1,NaN,NaN,1.0,1,326,10,1000,0.0
282,3,2017-03-08,7,NaN,NaN,1,NaN,NaN,1.0,2,326,10,1000,0.0
283,3,2017-03-09,7,NaN,NaN,1,NaN,NaN,1.0,3,326,10,1000,0.0
284,3,2017-03-10,7,NaN,NaN,1,NaN,NaN,1.0,4,326,10,1000,0.0
285,3,2017-03-11,7,NaN,NaN,0,NaN,NaN,1.0,5,326,10,1000,0.0


In [58]:
temp

,Month,Date,Region,NumberOfSales,NumberOfSales_lastmonth,HasPromotions,NumberOfSales_yesterday,NumberOfSales_lastweek,IsOpen_yesterday,DayOfWeek,NearestCompetitor,Week,StoreID,IsHoliday_tomorrow
278,3,2017-03-03,7,6646.896,NaN,0,0.000,NaN,1.0,4,326,9,1000,0.0
279,3,2017-03-04,7,6499.324,6646.896,0,6646.896,6646.896,1.0,5,326,9,1000,0.0
280,3,2017-03-06,7,10782.412,13146.220,1,0.000,13146.220,0.0,0,326,10,1000,0.0
281,3,2017-03-07,7,9094.596,23928.632,1,10782.412,23928.632,1.0,1,326,10,1000,0.0
282,3,2017-03-08,7,8313.472,33023.228,1,9094.596,33023.228,1.0,2,326,10,1000,0.0
283,3,2017-03-09,7,8084.752,41336.700,1,8313.472,41336.700,1.0,3,326,10,1000,0.0
284,3,2017-03-10,7,7990.780,49421.452,1,8084.752,49421.452,1.0,4,326,10,1000,0.0
285,3,2017-03-11,7,6467.024,57412.232,0,7990.780,50765.336,1.0,5,326,10,1000,0.0
286,3,2017-03-13,7,6760.676,63879.256,0,0.000,50733.036,0.0,0,326,11,1000,0.0
287,3,2017-03-14,7,6013.148,70639.932,0,6760.676,46711.300,1.0,1,326,11,1000,0.0


In [59]:
import datetime 
import dateutil.relativedelta
from IPython.display import clear_output

# prepare the dataframe where store the prediction
y_predicted = pd.DataFrame(np.nan, index=Xtest.index, columns=['NumberOfSales_predicted'])
    
for index, row in Xtest.iterrows():
    clear_output()
    print('processing... ' , index/df.index.count())
    
    # get from train set all the rows about this store
    temp = df_dates.loc[df_dates.StoreID == row['StoreID']]
    
    # get the date of the 'row' 
    date = temp.loc[index]
    date = date['Date']
    
    # get all the data from the last month
    splitdate = date + dateutil.relativedelta.relativedelta(days=-31)
    temp = temp[temp.Date >= splitdate]
    temp = temp[temp.Date <= date]
     
    temp.sort_values(by='Date')
    # switch index to timestamps to make this easier
    oldindex = temp.index
    temp.index = temp['Date']
    
    temp['NumberOfSales_yesterday'] = temp.NumberOfSales.rolling(window='1d',closed='left', min_periods=1).sum()
    temp['NumberOfSales_lastweek'] = temp.NumberOfSales.rolling(window='7d',closed='left', min_periods=1).sum()
    temp['NumberOfSales_lastmonth'] = temp.NumberOfSales.rolling(window='30d',closed='left', min_periods=1).sum()

    temp.NumberOfSales_yesterday = temp.NumberOfSales_yesterday.fillna(0)
    temp.NumberOfSales_lastweek = temp.NumberOfSales_lastweek.fillna(0)

    # put it back in the dataframe
    temp.index = oldindex
    
    # put the result in 'row'
    row['NumberOfSales_yesterday'] = temp.loc[index,'NumberOfSales_yesterday']
    row['NumberOfSales_lastweek'] = temp.loc[index,'NumberOfSales_lastweek']
    row['NumberOfSales_lastmonth'] = temp.loc[index,'NumberOfSales_lastmonth']

    # predict NumberOfSales of today
    temp.loc[index,'NumberOfSales'] = forest.predict(row.values.reshape(1, -1))
    
    # at the next iter my I can use this prediction for a new rolling
    df_dates.loc[index] = temp.loc[index]

    y_predicted.loc[index] = temp.loc[index,'NumberOfSales'] 



working on  276
working on  277
working on  278
working on  279
working on  280
working on  281
working on  282
working on  283
working on  284
working on  285
working on  286
working on  287
working on  288
working on  289
working on  290
working on  291
working on  292
working on  293
working on  294
working on  295
working on  296
working on  297
working on  298
working on  299
working on  300
working on  301
working on  302
working on  303
working on  304
working on  305
working on  306
working on  307
working on  308
working on  309
working on  310
working on  311
working on  312
working on  313
working on  314
working on  315
working on  316
working on  317
working on  318
working on  859
working on  860
working on  861
working on  862
working on  863
working on  864
working on  865
working on  866
working on  867
working on  868
working on  869
working on  870
working on  871
working on  872
working on  873
working on  874
working on  875
working on  876
working on  877
working 

working on  5507
working on  5508
working on  5509
working on  5510
working on  5511
working on  6065
working on  6066
working on  6067
working on  6068
working on  6069
working on  6070
working on  6071
working on  6072
working on  6073
working on  6074
working on  6075
working on  6076
working on  6077
working on  6078
working on  6079
working on  6080
working on  6081
working on  6082
working on  6083
working on  6084
working on  6085
working on  6086
working on  6087
working on  6088
working on  6089
working on  6090
working on  6091
working on  6092
working on  6093
working on  6094
working on  6095
working on  6096
working on  6097
working on  6098
working on  6099
working on  6100
working on  6101
working on  6102
working on  6103
working on  6104
working on  6105
working on  6106
working on  6107
working on  6108
working on  6109
working on  6110
working on  6111
working on  6112
working on  6113
working on  6114
working on  6668
working on  6669
working on  6670
working on  66

working on  11371
working on  11372
working on  11373
working on  11374
working on  11375
working on  11376
working on  11377
working on  11378
working on  11379
working on  11380
working on  11381
working on  11382
working on  11383
working on  11384
working on  11385
working on  11386
working on  11387
working on  11388
working on  11389
working on  11390
working on  11391
working on  11392
working on  11393
working on  11394
working on  11395
working on  11396
working on  11397
working on  11398
working on  11399
working on  11400
working on  11955
working on  11956
working on  11957
working on  11958
working on  11959
working on  11960
working on  11961
working on  11962
working on  11963
working on  11964
working on  11965
working on  11966
working on  11967
working on  11968
working on  11969
working on  11970
working on  11971
working on  11972
working on  11973
working on  11974
working on  11975
working on  11976
working on  11977
working on  11978
working on  11979
working on

working on  16813
working on  16814
working on  16815
working on  16816
working on  16817
working on  16818
working on  16819
working on  16820
working on  16821
working on  16822
working on  16823
working on  16824
working on  16825
working on  16826
working on  16827
working on  16828
working on  16829
working on  16830
working on  16831
working on  16832
working on  16833
working on  16834
working on  16835
working on  16836
working on  17391
working on  17392
working on  17393
working on  17394
working on  17395
working on  17396
working on  17397
working on  17398
working on  17399
working on  17400
working on  17401
working on  17402
working on  17403
working on  17404
working on  17405
working on  17406
working on  17407
working on  17408
working on  17409
working on  17410
working on  17411
working on  17412
working on  17413
working on  17414
working on  17415
working on  17416
working on  17417
working on  17418
working on  17419
working on  17420
working on  17421
working on

working on  22092
working on  22093
working on  22094
working on  22095
working on  22096
working on  22097
working on  22098
working on  22099
working on  22100
working on  22101
working on  22102
working on  22103
working on  22104
working on  22105
working on  22106
working on  22107
working on  22108
working on  22109
working on  22653
working on  22654
working on  22655
working on  22656
working on  22657
working on  22658
working on  22659
working on  22660
working on  22661
working on  22662
working on  22663
working on  22664
working on  22665
working on  22666
working on  22667
working on  22668
working on  22669
working on  22670
working on  22671
working on  22672
working on  22673
working on  22674
working on  22675
working on  22676
working on  22677
working on  22678
working on  22679
working on  22680
working on  22681
working on  22682
working on  22683
working on  22684
working on  22685
working on  22686
working on  22687
working on  22688
working on  22689
working on

...

working on  405102
working on  405103
working on  405104
working on  405105
working on  405106
working on  405107
working on  405108
working on  405109
working on  405110
working on  405111
working on  405112
working on  405113
working on  405114
working on  405115
working on  405116
working on  405117
working on  405118
working on  405119
working on  405675
working on  405676
working on  405677
working on  405678
working on  405679
working on  405680
working on  405681
working on  405682
working on  405683
working on  405684
working on  405685
working on  405686
working on  405687
working on  405688
working on  405689
working on  405690
working on  405691
working on  405692
working on  405693
working on  405694
working on  405695
working on  405696
working on  405697
working on  405698
working on  405699
working on  405700
working on  405701
working on  405702
working on  405703
working on  405704
working on  405705
working on  405706
working on  405707
working on  405708
working on  

working on  410370
working on  410371
working on  410372
working on  410373
working on  410374
working on  410375
working on  410376
working on  410377
working on  410378
working on  410379
working on  410380
working on  410381
working on  410382
working on  410383
working on  410384
working on  410385
working on  410386
working on  410387
working on  410388
working on  410389
working on  410390
working on  410391
working on  410392
working on  410393
working on  410394
working on  410395
working on  410396
working on  410397
working on  410398
working on  410399
working on  410400
working on  410401
working on  410402
working on  410403
working on  410404
working on  410405
working on  410805
working on  410806
working on  410807
working on  410808
working on  410809
working on  410810
working on  410811
working on  410812
working on  410813
working on  410814
working on  410815
working on  410816
working on  410817
working on  410818
working on  410819
working on  410820
working on  

working on  414925
working on  414926
working on  414927
working on  415479
working on  415480
working on  415481
working on  415482
working on  415483
working on  415484
working on  415485
working on  415486
working on  415487
working on  415488
working on  415489
working on  415490
working on  415491
working on  415492
working on  415493
working on  415494
working on  415495
working on  415496
working on  415497
working on  415498
working on  415499
working on  415500
working on  415501
working on  415502
working on  415503
working on  415504
working on  415505
working on  415506
working on  415507
working on  415508
working on  415509
working on  415510
working on  415511
working on  415512
working on  415513
working on  415514
working on  415515
working on  415516
working on  415517
working on  415518
working on  415519
working on  415520
working on  415521
working on  415522
working on  415523
working on  415524
working on  415525
working on  415526
working on  415527
working on  

working on  420101
working on  420102
working on  420103
working on  420104
working on  420105
working on  420106
working on  420107
working on  420108
working on  420109
working on  420110
working on  420111
working on  420112
working on  420113
working on  420114
working on  420115
working on  420116
working on  420117
working on  420118
working on  420119
working on  420120
working on  420121
working on  420122
working on  420123
working on  420124
working on  420125
working on  420126
working on  420127
working on  420128
working on  420682
working on  420683
working on  420684
working on  420685
working on  420686
working on  420687
working on  420688
working on  420689
working on  420690
working on  420691
working on  420692
working on  420693
working on  420694
working on  420695
working on  420696
working on  420697
working on  420698
working on  420699
working on  420700
working on  420701
working on  420702
working on  420703
working on  420704
working on  420705
working on  

working on  425504
working on  425505
working on  425506
working on  425507
working on  425508
working on  425509
working on  425510
working on  425511
working on  425512
working on  425513
working on  425514
working on  425515
working on  425516
working on  425517
working on  425518
working on  425519
working on  425520
working on  425521
working on  425522
working on  425523
working on  425524
working on  425525
working on  425526
working on  425527
working on  425528
working on  425529
working on  425530
working on  425531
working on  425532
working on  425533
working on  425534
working on  425535
working on  425536
working on  425537
working on  425538
working on  425539
working on  425540
working on  425541
working on  425542
working on  425543
working on  425544
working on  425545
working on  425546
working on  425547
working on  425548


## Score bipmeter

In [ ]:
# FOR DEBUGGING PORPOSE keep only one region in the test set
df_test = df_test[df_test.Region == 4]

In [60]:
df_test[['Date','Month','Region','StoreID','NumberOfSales']]

,Date,Month,Region,StoreID,NumberOfSales
276,2017-03-01,3,7,1000,5509.0
277,2017-03-02,3,7,1000,5881.0
278,2017-03-03,3,7,1000,8463.0
279,2017-03-04,3,7,1000,8146.0
280,2017-03-06,3,7,1000,11306.0
281,2017-03-07,3,7,1000,9843.0
282,2017-03-08,3,7,1000,8049.0
283,2017-03-09,3,7,1000,8128.0
284,2017-03-10,3,7,1000,8251.0
285,2017-03-11,3,7,1000,6754.0


In [61]:
df1 = df_test.join(y_predicted)
df1 = df1.groupby(['Month','StoreID','Region'], as_index=False).agg({"NumberOfSales": "sum","NumberOfSales_predicted":"sum"})
df1['Error'] = abs(df1['NumberOfSales_predicted']-df1['NumberOfSales'])
E_r = df1.groupby('Region', as_index=False).agg({"Error":"sum","NumberOfSales":"sum"})

score_r = E_r['Error']/E_r['NumberOfSales']
score_r

0     0.060927
1     0.064622
2     0.044447
3     0.045699
4     0.048160
5     0.064306
6     0.065505
7     0.060984
8     0.049679
9     0.052148
10    0.065788
dtype: float64

### Final Score

In [64]:
score_r.sum()/score_r.count()

0.056569690477512015

In [65]:
df1

,Month,StoreID,Region,NumberOfSales,NumberOfSales_predicted,Error
0,3,1000,7,152820.0,155701.708,2881.708
1,3,1001,0,70708.0,74283.240,3575.240
2,3,1002,3,127779.0,127618.680,160.320
3,3,1003,7,129289.0,139126.832,9837.832
4,3,1004,2,100886.0,100839.900,46.100
5,3,1005,9,124437.0,123039.192,1397.808
6,3,1006,10,165932.0,157851.580,8080.420
7,3,1007,10,99084.0,93866.948,5217.052
8,3,1008,9,168177.0,164823.852,3353.148
9,3,1009,0,130890.0,128361.972,2528.028


## Score  ( deprecated ) - use bip ufficial method

In [246]:
df1 = (df_test.join(y_predicted))

In [248]:
# group by month & region 
result=df1.groupby(['Month','Region'], as_index=False).agg({"NumberOfSales": "sum","NumberOfSales_predicted":"sum"})

# % error
result['% Error']=abs(result['NumberOfSales']-result['NumberOfSales_predicted'])/result['NumberOfSales']*100

In [249]:
result

,Month,Region,NumberOfSales,NumberOfSales_predicted,% Error
0,1,3,13336732.0,1.608165e+07,20.581613
1,2,3,12419835.0,1.517083e+07,22.149978
2,11,3,1195791.0,1.125130e+06,5.909127
3,12,3,15320495.0,1.640271e+07,7.063807


In [252]:
result['% Error'].sum()/result.count().max()

13.926131181847682